# Import libraries

In [1]:
from google.colab import drive
drive.mount("/content/drive/")

import sys
sys.path.insert(0,'/content/drive/MyDrive/')

Mounted at /content/drive/


In [2]:
cd /content/drive/MyDrive/ROBT613_BMI_Final_Project

/content/drive/MyDrive/ROBT613_BMI_Final_Project


In [3]:
!pip install mne==0.18.0

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.3/6.3 MB 8.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for mne: filename=mne-0.18.0-py3-none-any.whl size=6313874 sha256=3a6d2db5dfa1c0ae16da9d8e1c8e8967bfde23df7f0e06d3f556eb7294a18c8f
  Stored in directory: /root/.cache/pip/wheels/50/55/a4/d95e3c6ab9370b77f9c871e9882a9578a2195c64ede88694ff
Successfully built mne


In [4]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import TensorDataset, DataLoader
from torch.autograd import Variable

import itertools
import pandas as pd
import pickle
import numpy as np
import mne
from sklearn.preprocessing import StandardScaler

from nu_smrutils import loaddat,subject_specific,augment_dataset,crop_data,train_model

%matplotlib inline
%load_ext autoreload
%autoreload 2
torch.manual_seed(0)

# Select device

In [5]:
#by default
dev = torch.device("cuda" if torch.cuda.is_available() else "cpu")

if dev.type == 'cuda':
   print('Your GPU device name :', torch.cuda.get_device_name())

Your GPU device name : Tesla T4


# Select Dataset

## 1. BNCI2014004

In [6]:
dname = dict(BNCI2014004 = 'aBNCI2014004R.pickle',
             BNCI2014001 = 'aBNCI2014001R.pickle',
             Weibo2014   = 'aWeibo2014R.pickle',
             Physionet   = 'aPhysionetRR.pickle')

itemname = 'BNCI2014004'

filename = dname[itemname]
iname = itemname + '__'

data = loaddat(filename)
print('Number of subjects in data :', len(data))

Number of subjects in data : 9


# Load subject specific data, preprocess data (filtering, data augmentation, data cropping, standard scaling), and use Pytorch dataloaders

Function for standard scaler

In [7]:
scaler = StandardScaler()
def standard_scaling(data):
  shape1 = data.shape[1]
  shape2 = data.shape[2]
  data = data.reshape(data.shape[0],-1)
  data = scaler.fit_transform(data)
  data = data.reshape(data.shape[0],shape1,shape2)
  return data

Function for Pytorch dataloaders

In [8]:
def get_data_loaders(dat, batch_size, EEGNET = None, dev=dev):
    # convert data dimensions to into to gray scale image format
    if EEGNET: ### EEGNet model requires the last dimension to be 1
        ff = lambda dat: torch.unsqueeze(dat, dim = -1)
    else:
        ff = lambda dat: torch.unsqueeze(dat, dim = 1)

    x_train, x_valid, x_test =  map(ff,(dat['xtrain'],dat['xvalid'],dat['xtest']))
    y_train, y_valid, y_test =  dat['ytrain'], dat['yvalid'], dat['ytest']
    print('Input data shape', x_train.shape)

    # TensorDataset & Dataloader
    train_dat = TensorDataset(x_train.to(dev), y_train.to(dev))
    val_dat   = TensorDataset(x_valid.to(dev), y_valid.to(dev))

    train_loader = DataLoader(train_dat, batch_size = batch_size, shuffle = True)
    val_loader   = DataLoader(val_dat,   batch_size = batch_size, shuffle = False)

    output = dict(dset_loaders = {'train': train_loader, 'val': val_loader},
                  dset_sizes  =  {'train': len(x_train), 'val': len(x_valid)},
                  test_data   =  {'x_test' : x_test, 'y_test' : y_test})
    return output

In [9]:
#Filter
print('\nFiltering from 8 to 15 Hz')
for i in range(len(data)):
  data[i].filter(l_freq = 8, h_freq = 15)

#--------------------------------------------------------------------------
#Subject specific
subjectIndex = list(range(108))
class_name = ['left_hand', 'right_hand']
print('\nLoading subject specific data')
datt = subject_specific(data, subjectIndex, class_name, test_size = 0.15)


#--------------------------------------------------------------------------
# augment only the training data
print('\nData augmentation')
augmdata = dict(std_dev = 0.00001,  multiple = 2)

for ii, dat in enumerate(datt):
    xtrain, ytrain = augment_dataset(dat['xtrain'], dat['ytrain'],
                                     std_dev = augmdata['std_dev'],
                                     multiple = augmdata['multiple'])

    print("Shape after data augmentation :", xtrain.shape)
    datt[ii]['xtrain'], datt[ii]['ytrain'] = xtrain, ytrain

#--------------------------------------------------------------------------
# Data Cropping
print('\nData cropping')
fs = 80 # sampling frequency
crop_len = 1 #or None
crop = dict(fs = fs, crop_len = crop_len)

for ii, dat in enumerate(datt):
    if crop['crop_len']:
       X_train, y_train = crop_data(crop['fs'], crop['crop_len'], dat['xtrain'],
                                    dat['ytrain'],xpercent = 50)
       X_valid, y_valid = crop_data(crop['fs'], crop['crop_len'], dat['xvalid'],
                                    dat['yvalid'],xpercent = 50)
       X_test,  y_test  = crop_data(crop['fs'], crop['crop_len'], dat['xtest'],
                                    dat['ytest'], xpercent = 50)

       print(X_train.shape)
       datt[ii] = dict(xtrain = X_train, xvalid = X_valid, xtest = X_test,
                       ytrain = y_train, yvalid = y_valid, ytest = y_test)

#--------------------------------------------------------------------------
# Standard scaling
print('\nPreprocessing')
for i in range(0,len(datt)):
  datt[i]['xtrain']=standard_scaling(datt[i]['xtrain'])
  datt[i]['xvalid']=standard_scaling(datt[i]['xvalid'])
  datt[i]['xtest']=standard_scaling(datt[i]['xtest'])
  datt[i]['xtrain']=torch.tensor(datt[i]['xtrain'])
  datt[i]['xvalid']=torch.tensor(datt[i]['xvalid'])
  datt[i]['xtest']=torch.tensor(datt[i]['xtest'])
  datt[i]['xtrain']=datt[i]['xtrain'].type('torch.FloatTensor')
  datt[i]['xvalid']=datt[i]['xvalid'].type('torch.FloatTensor')
  datt[i]['xtest']=datt[i]['xtest'].type('torch.FloatTensor')

#--------------------------------------------------------------------------
# Dataloaders
print('\nDataloaders')
for ii, dat in enumerate(datt):
    print('Subjects :', ii+1)
    datt[ii] = get_data_loaders(dat, batch_size = 64)


Filtering from 8 to 15 Hz


/usr/local/lib/python3.10/dist-packages/mne/io/pick.py:360: DeprecationWarning: `np.bool` is a deprecated alias for the builtin `bool`. To silence this warning, use `bool` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.bool_` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  pick = np.zeros(nchan, dtype=np.bool)
/usr/local/lib/python3.10/dist-packages/mne/io/pick.py:360: DeprecationWarning: `np.bool` is a deprecated alias for the builtin `bool`. To silence this warning, use `bool` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.bool_` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  pick = np.zeros(nchan, dtype=np.bool)
/usr/local/lib/python3.10/dist-packages/mne/filter.py:312: Dep


Loading subject specific data
Loading subject: 0
Loading subject: 1
Loading subject: 2
Loading subject: 3
Loading subject: 4
Loading subject: 5
Loading subject: 6
Loading subject: 7
Loading subject: 8
Loading subject: 9
list index out of range

Data augmentation
Shape after data augmentation : torch.Size([1040, 3, 321])
Shape after data augmentation : torch.Size([982, 3, 321])
Shape after data augmentation : torch.Size([1040, 3, 321])
Shape after data augmentation : torch.Size([1068, 3, 321])
Shape after data augmentation : torch.Size([1068, 3, 321])
Shape after data augmentation : torch.Size([1040, 3, 321])
Shape after data augmentation : torch.Size([1040, 3, 321])
Shape after data augmentation : torch.Size([1098, 3, 321])
Shape after data augmentation : torch.Size([1040, 3, 321])

Data cropping
torch.Size([3120, 3, 120])
torch.Size([2946, 3, 120])
torch.Size([3120, 3, 120])
torch.Size([3204, 3, 120])
torch.Size([3204, 3, 120])
torch.Size([3120, 3, 120])
torch.Size([3120, 3, 120])
to

# CNN architecture




In [10]:
class CNN_small(nn.Module):
    def __init__(self, input_size):
        super(CNN_small, self).__init__()
        # convolutional layer (sees 1x16x76 image tensor)
        self.features = nn.Sequential(
            nn.Conv2d(1, 32, 3, padding=1),
            nn.BatchNorm2d(32),
            nn.LeakyReLU(),
            nn.MaxPool2d((1,2), (1, 2)),

            nn.Conv2d(32, 64, 3, padding=1),
            nn.BatchNorm2d(64),
            nn.LeakyReLU(),
            nn.MaxPool2d((1,2), (1, 2)))


        self.flat_fts = self.get_out_dim(input_size, self.features)

        self.classifier = nn.Sequential(
            nn.Linear(self.flat_fts, 400),
            nn.Dropout(0.25),
            nn.Linear(400, 2))

    def get_out_dim(self, in_size, fts):
        with torch.no_grad():
            f = fts(Variable(torch.ones(1, *in_size)))
            out = int(np.prod(f.size()[1:]))
            return out

    def forward(self, x):
        # add sequence of convolutional and max pooling layers
        fts = self.features(x)
        flat_fts = fts.view(-1, self.flat_fts)
        return self.classifier(flat_fts)


# Training loop

In [11]:
#Hyperparameter settings
num_epochs = 150
learning_rate = 1e-4
weight_decay = 1e-4
batch_size = 64
verbose = 2


#% used to save the results table
results = {}
table = pd.DataFrame(columns = ['Train_Acc', 'Val_Acc', 'Test_Acc', 'Epoch'])
sum=0
num=0
max=0

for i in range(len(datt)):
  subject = i
  print('----------------Subject '+str(subject+1) + '---------------------')
  description = 'Subject'+str(subject+1)
  dat = datt[subject]

  #% get input size (channel x timepoints)
  input_size = (1, dat['test_data']['x_test'].shape[-2],
                 dat['test_data']['x_test'].shape[-1])

  model = CNN_small(input_size)

  # optimizer and the loss function definition
  optimizer = torch.optim.Adam(model.parameters(),
                             lr = learning_rate,
                             weight_decay = weight_decay)
  criterion = torch.nn.CrossEntropyLoss()

  model.to(dev)
  criterion.to(dev)


  #******** Training loop *********
  best_model, train_losses, val_losses, train_accs, val_accs, info =\
      train_model(model, dat['dset_loaders'], dat['dset_sizes'],
                  criterion, optimizer, dev=dev, lr_scheduler=False,
                  num_epochs = num_epochs, verbose = verbose)


  test_samples = 50
  x_test = dat['test_data']['x_test'][:test_samples,:,:]
  y_test = dat['test_data']['y_test'][:test_samples]

  # predict test data
  preds = best_model(x_test.to(dev))
  preds_class = preds.data.max(1)[1]

  # get the accuracy
  corrects = torch.sum(preds_class == y_test.data.to(dev))
  test_acc = corrects.cpu().numpy()/x_test.shape[0]
  print("Test Accuracy :", test_acc)
  sum+=test_acc
  num+=1
  if test_acc>max:
    max=test_acc
  # save results
  tab = dict(Train_Acc= train_accs[info['best_epoch']],
            Val_Acc  = val_accs[info['best_epoch']],
            Test_Acc = test_acc, Epoch = info['best_epoch'] + 1)

  table.loc[description] = tab
  val_acc = np.max(val_accs)
  print('\n---------------------------------------------------------------------\n')

----------------Subject 1---------------------
Epoch 1/150
train loss: 0.0124, acc: 0.4904
val loss: 0.0124, acc: 0.5652
Epoch 2/150
train loss: 0.0108, acc: 0.5766
val loss: 0.0150, acc: 0.5181
Epoch 3/150
train loss: 0.0106, acc: 0.5888
val loss: 0.0127, acc: 0.5543
Epoch 4/150
train loss: 0.0094, acc: 0.6558
val loss: 0.0125, acc: 0.5543
Epoch 5/150
train loss: 0.0089, acc: 0.6872
val loss: 0.0133, acc: 0.6051
Epoch 6/150
train loss: 0.0083, acc: 0.7247
val loss: 0.0137, acc: 0.5326
Epoch 7/150
train loss: 0.0083, acc: 0.7067
val loss: 0.0139, acc: 0.5543
Epoch 8/150
train loss: 0.0073, acc: 0.7769
val loss: 0.0135, acc: 0.6232
Epoch 9/150
train loss: 0.0071, acc: 0.7708
val loss: 0.0137, acc: 0.5725
Epoch 10/150
train loss: 0.0065, acc: 0.7939
val loss: 0.0139, acc: 0.5797
Epoch 11/150
train loss: 0.0062, acc: 0.8128
val loss: 0.0168, acc: 0.5978
Epoch 12/150
train loss: 0.0059, acc: 0.8154
val loss: 0.0152, acc: 0.5978
Epoch 13/150
train loss: 0.0055, acc: 0.8362
val loss: 0.0168,

# Save and print results

In [12]:
table.to_csv('results/CNN_small/BNCI2014004_cnn_small.csv')

print(table)
print(f"\nAverage between subjects: {(sum/num):.2f}")
print(f"Max test_score between subjects: {max:.2f}")

          Train_Acc   Val_Acc  Test_Acc  Epoch
Subject1   0.892949  0.637681      0.64     31
Subject2   0.581806  0.582375      0.40      3
Subject3   0.848077  0.536232      0.66     16
Subject4   0.801186  0.887719      0.96      5
Subject5   0.870162  0.680702      0.58     25
Subject6   0.957051  0.695652      0.64     43
Subject7   0.892308  0.677536      0.64     26
Subject8   0.910443  0.800687      0.78     20
Subject9   0.836859  0.702899      0.74     11

Average between subjects: 0.67
Max test_score between subjects: 0.96


Repeat the same steps for other datasets
## 2. BNCI2014001

In [13]:
itemname = 'BNCI2014001'

filename = dname[itemname]
iname = itemname + '__'

data = loaddat(filename)
print('Number of subjects in data :', len(data))

Number of subjects in data : 9


In [14]:
#Filter
print('\nFiltering from 8 to 15 Hz')
for i in range(len(data)):
  data[i].filter(l_freq = 8, h_freq = 15)

#--------------------------------------------------------------------------
#Subject specific
subjectIndex = list(range(108))
class_name = ['left_hand', 'right_hand']
print('\nLoading subject specific data')
datt = subject_specific(data, subjectIndex, class_name, test_size = 0.15)


#--------------------------------------------------------------------------
# augment only the training data
print('\nData augmentation')
augmdata = dict(std_dev = 0.000001,  multiple = 2)

for ii, dat in enumerate(datt):
    xtrain, ytrain = augment_dataset(dat['xtrain'], dat['ytrain'],
                                     std_dev = augmdata['std_dev'],
                                     multiple = augmdata['multiple'])

    print("Shape after data augmentation :", xtrain.shape)
    datt[ii]['xtrain'], datt[ii]['ytrain'] = xtrain, ytrain

#--------------------------------------------------------------------------
# Data Cropping
print('\nData cropping')
fs = 80 # sampling frequency
crop_len = 1 #or None
crop = dict(fs = fs, crop_len = crop_len)

for ii, dat in enumerate(datt):
    if crop['crop_len']:
       X_train, y_train = crop_data(crop['fs'], crop['crop_len'], dat['xtrain'],
                                    dat['ytrain'],xpercent = 50)
       X_valid, y_valid = crop_data(crop['fs'], crop['crop_len'], dat['xvalid'],
                                    dat['yvalid'],xpercent = 50)
       X_test,  y_test  = crop_data(crop['fs'], crop['crop_len'], dat['xtest'],
                                    dat['ytest'], xpercent = 50)

       print(X_train.shape)
       datt[ii] = dict(xtrain = X_train, xvalid = X_valid, xtest = X_test,
                       ytrain = y_train, yvalid = y_valid, ytest = y_test)

#--------------------------------------------------------------------------
# Standard scaling
print('\nPreprocessing')
for i in range(0,len(datt)):
  datt[i]['xtrain']=standard_scaling(datt[i]['xtrain'])
  datt[i]['xvalid']=standard_scaling(datt[i]['xvalid'])
  datt[i]['xtest']=standard_scaling(datt[i]['xtest'])
  datt[i]['xtrain']=torch.tensor(datt[i]['xtrain'])
  datt[i]['xvalid']=torch.tensor(datt[i]['xvalid'])
  datt[i]['xtest']=torch.tensor(datt[i]['xtest'])
  datt[i]['xtrain']=datt[i]['xtrain'].type('torch.FloatTensor')
  datt[i]['xvalid']=datt[i]['xvalid'].type('torch.FloatTensor')
  datt[i]['xtest']=datt[i]['xtest'].type('torch.FloatTensor')

#--------------------------------------------------------------------------
# Dataloaders
print('\nDataloaders')
for ii, dat in enumerate(datt):
    print('Subjects :', ii+1)
    datt[ii] = get_data_loaders(dat, batch_size = 64)


Filtering from 8 to 15 Hz


/usr/local/lib/python3.10/dist-packages/mne/io/pick.py:360: DeprecationWarning: `np.bool` is a deprecated alias for the builtin `bool`. To silence this warning, use `bool` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.bool_` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  pick = np.zeros(nchan, dtype=np.bool)
/usr/local/lib/python3.10/dist-packages/mne/io/pick.py:360: DeprecationWarning: `np.bool` is a deprecated alias for the builtin `bool`. To silence this warning, use `bool` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.bool_` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  pick = np.zeros(nchan, dtype=np.bool)
/usr/local/lib/python3.10/dist-packages/mne/filter.py:312: Dep


Loading subject specific data
Loading subject: 0
Loading subject: 1
Loading subject: 2
Loading subject: 3
Loading subject: 4
Loading subject: 5
Loading subject: 6
Loading subject: 7
Loading subject: 8
Loading subject: 9
list index out of range

Data augmentation
Shape after data augmentation : torch.Size([414, 22, 321])
Shape after data augmentation : torch.Size([414, 22, 321])
Shape after data augmentation : torch.Size([414, 22, 321])
Shape after data augmentation : torch.Size([414, 22, 321])
Shape after data augmentation : torch.Size([414, 22, 321])
Shape after data augmentation : torch.Size([414, 22, 321])
Shape after data augmentation : torch.Size([414, 22, 321])
Shape after data augmentation : torch.Size([414, 22, 321])
Shape after data augmentation : torch.Size([414, 22, 321])

Data cropping
torch.Size([1242, 22, 120])
torch.Size([1242, 22, 120])
torch.Size([1242, 22, 120])
torch.Size([1242, 22, 120])
torch.Size([1242, 22, 120])
torch.Size([1242, 22, 120])
torch.Size([1242, 22, 

In [15]:
#Hyperparameter settings
num_epochs = 150
learning_rate = 1e-3
weight_decay = 1e-3
batch_size = 64
verbose = 2


#% used to save the results table
results = {}
table = pd.DataFrame(columns = ['Train_Acc', 'Val_Acc', 'Test_Acc', 'Epoch'])
sum=0
num=0
max=0

for i in range(len(datt)):
  subject = i
  print('----------------Subject '+str(subject+1) + '---------------------')
  description = 'Subject'+str(subject+1)
  dat = datt[subject]

  #% get input size (channel x timepoints)
  input_size = (1, dat['test_data']['x_test'].shape[-2],
                 dat['test_data']['x_test'].shape[-1])

  model = CNN_small(input_size)

  # optimizer and the loss function definition
  optimizer = torch.optim.Adam(model.parameters(),
                             lr = learning_rate,
                             weight_decay = weight_decay)
  criterion = torch.nn.CrossEntropyLoss()

  model.to(dev)
  criterion.to(dev)

  #******** Training loop *********
  best_model, train_losses, val_losses, train_accs, val_accs, info =\
      train_model(model, dat['dset_loaders'], dat['dset_sizes'],
                  criterion, optimizer, dev=dev, lr_scheduler=False,
                  num_epochs = num_epochs, verbose = verbose)


  test_samples = 50
  x_test = dat['test_data']['x_test'][:test_samples,:,:]
  y_test = dat['test_data']['y_test'][:test_samples]

  # predict test data
  preds = best_model(x_test.to(dev))
  preds_class = preds.data.max(1)[1]

  # get the accuracy
  corrects = torch.sum(preds_class == y_test.data.to(dev))
  test_acc = corrects.cpu().numpy()/x_test.shape[0]
  print("Test Accuracy :", test_acc)
  sum+=test_acc
  num+=1
  if test_acc>max:
    max=test_acc
  # save results
  tab = dict(Train_Acc= train_accs[info['best_epoch']],
            Val_Acc  = val_accs[info['best_epoch']],
            Test_Acc = test_acc, Epoch = info['best_epoch'] + 1)

  table.loc[description] = tab
  val_acc = np.max(val_accs)
  print('\n---------------------------------------------------------------------\n')

----------------Subject 1---------------------
Epoch 1/150
train loss: 0.3949, acc: 0.4984
val loss: 0.1632, acc: 0.5315
Epoch 2/150
train loss: 0.0700, acc: 0.6248
val loss: 0.0446, acc: 0.5045
Epoch 3/150
train loss: 0.0175, acc: 0.7206
val loss: 0.0359, acc: 0.5405
Epoch 4/150
train loss: 0.0085, acc: 0.7987
val loss: 0.0287, acc: 0.5405
Epoch 5/150
train loss: 0.0066, acc: 0.8414
val loss: 0.0300, acc: 0.5676
Epoch 6/150
train loss: 0.0053, acc: 0.8591
val loss: 0.0286, acc: 0.5856
Epoch 7/150
train loss: 0.0033, acc: 0.9243
val loss: 0.0288, acc: 0.5586
Epoch 8/150
train loss: 0.0029, acc: 0.9235
val loss: 0.0296, acc: 0.5586
Epoch 9/150
train loss: 0.0017, acc: 0.9646
val loss: 0.0269, acc: 0.5856
Epoch 10/150
train loss: 0.0009, acc: 0.9847
val loss: 0.0262, acc: 0.5946
Epoch 11/150
train loss: 0.0006, acc: 0.9936
val loss: 0.0274, acc: 0.5856
Epoch 12/150
train loss: 0.0003, acc: 0.9992
val loss: 0.0266, acc: 0.5766
Epoch 13/150
train loss: 0.0002, acc: 1.0000
val loss: 0.0268,

In [16]:
table.to_csv('results/CNN_small/BNCI2014001_cnn_small.csv')

print(table)
print(f"\nAverage between subjects: {(sum/num):.2f}")
print(f"Max test_score between subjects: {max:.2f}")

          Train_Acc   Val_Acc  Test_Acc  Epoch
Subject1   1.000000  0.639640      0.76    148
Subject2   1.000000  0.612613      0.58     72
Subject3   0.746377  0.828829      0.78      2
Subject4   0.517713  0.603604      0.46    138
Subject5   0.512077  0.549550      0.56      1
Subject6   0.826892  0.603604      0.50     88
Subject7   0.585346  0.558559      0.50      2
Subject8   1.000000  0.801802      0.90     80
Subject9   0.991948  0.828829      0.84      7

Average between subjects: 0.65
Max test_score between subjects: 0.90


## 3. Weibo2014

In [17]:
itemname = 'Weibo2014'

filename = dname[itemname]
iname = itemname + '__'

data = loaddat(filename)
print('Number of subjects in data :', len(data))

Number of subjects in data : 10


In [18]:
#Filter
print('\nFiltering from 8 to 15 Hz')
for i in range(len(data)):
  data[i].filter(l_freq = 8, h_freq = 15)

#--------------------------------------------------------------------------
#Subject specific
subjectIndex = list(range(108))
class_name = ['left_hand', 'right_hand']
print('\nLoading subject specific data')
datt = subject_specific(data, subjectIndex, class_name, test_size = 0.15)


#--------------------------------------------------------------------------
# augment only the training data
print('\nData augmentation')
augmdata = dict(std_dev = 0.00001,  multiple = 2)

for ii, dat in enumerate(datt):
    xtrain, ytrain = augment_dataset(dat['xtrain'], dat['ytrain'],
                                     std_dev = augmdata['std_dev'],
                                     multiple = augmdata['multiple'])

    print("Shape after data augmentation :", xtrain.shape)
    datt[ii]['xtrain'], datt[ii]['ytrain'] = xtrain, ytrain

#--------------------------------------------------------------------------
# Data Cropping
print('\nData cropping')
fs = 80 # sampling frequency
crop_len = 1 #or None
crop = dict(fs = fs, crop_len = crop_len)

for ii, dat in enumerate(datt):
    if crop['crop_len']:
       X_train, y_train = crop_data(crop['fs'], crop['crop_len'], dat['xtrain'],
                                    dat['ytrain'],xpercent = 50)
       X_valid, y_valid = crop_data(crop['fs'], crop['crop_len'], dat['xvalid'],
                                    dat['yvalid'],xpercent = 50)
       X_test,  y_test  = crop_data(crop['fs'], crop['crop_len'], dat['xtest'],
                                    dat['ytest'], xpercent = 50)

       print(X_train.shape)
       datt[ii] = dict(xtrain = X_train, xvalid = X_valid, xtest = X_test,
                       ytrain = y_train, yvalid = y_valid, ytest = y_test)

#--------------------------------------------------------------------------
# Standard scaling
print('\nPreprocessing')
for i in range(0,len(datt)):
  datt[i]['xtrain']=standard_scaling(datt[i]['xtrain'])
  datt[i]['xvalid']=standard_scaling(datt[i]['xvalid'])
  datt[i]['xtest']=standard_scaling(datt[i]['xtest'])
  datt[i]['xtrain']=torch.tensor(datt[i]['xtrain'])
  datt[i]['xvalid']=torch.tensor(datt[i]['xvalid'])
  datt[i]['xtest']=torch.tensor(datt[i]['xtest'])
  datt[i]['xtrain']=datt[i]['xtrain'].type('torch.FloatTensor')
  datt[i]['xvalid']=datt[i]['xvalid'].type('torch.FloatTensor')
  datt[i]['xtest']=datt[i]['xtest'].type('torch.FloatTensor')

#--------------------------------------------------------------------------
# Dataloaders
print('\nDataloaders')
for ii, dat in enumerate(datt):
    print('Subjects :', ii+1)
    datt[ii] = get_data_loaders(dat, batch_size = 64)


Filtering from 8 to 15 Hz


/usr/local/lib/python3.10/dist-packages/mne/io/pick.py:360: DeprecationWarning: `np.bool` is a deprecated alias for the builtin `bool`. To silence this warning, use `bool` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.bool_` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  pick = np.zeros(nchan, dtype=np.bool)
/usr/local/lib/python3.10/dist-packages/mne/io/pick.py:360: DeprecationWarning: `np.bool` is a deprecated alias for the builtin `bool`. To silence this warning, use `bool` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.bool_` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  pick = np.zeros(nchan, dtype=np.bool)
/usr/local/lib/python3.10/dist-packages/mne/filter.py:312: Dep


Loading subject specific data
Loading subject: 0
Loading subject: 1
Loading subject: 2
Loading subject: 3
Loading subject: 4
Loading subject: 5
Loading subject: 6
Loading subject: 7
Loading subject: 8
Loading subject: 9
Loading subject: 10
list index out of range

Data augmentation
Shape after data augmentation : torch.Size([230, 60, 321])
Shape after data augmentation : torch.Size([230, 60, 321])
Shape after data augmentation : torch.Size([230, 60, 321])
Shape after data augmentation : torch.Size([230, 60, 321])
Shape after data augmentation : torch.Size([230, 60, 321])
Shape after data augmentation : torch.Size([202, 60, 321])
Shape after data augmentation : torch.Size([230, 60, 321])
Shape after data augmentation : torch.Size([230, 60, 321])
Shape after data augmentation : torch.Size([230, 60, 321])
Shape after data augmentation : torch.Size([230, 60, 321])

Data cropping
torch.Size([690, 60, 120])
torch.Size([690, 60, 120])
torch.Size([690, 60, 120])
torch.Size([690, 60, 120])
tor

In [19]:
#Hyperparameter settings
num_epochs = 150
learning_rate = 1e-3
weight_decay = 1e-4
batch_size = 64
verbose = 2


#% used to save the results table
results = {}
table = pd.DataFrame(columns = ['Train_Acc', 'Val_Acc', 'Test_Acc', 'Epoch'])
sum=0
num=0
max=0

for i in range(len(datt)):
  subject = i
  print('----------------Subject '+str(subject+1) + '---------------------')
  description = 'Subject'+str(subject+1)
  dat = datt[subject]

  #% get input size (channel x timepoints)
  input_size = (1, dat['test_data']['x_test'].shape[-2],
                 dat['test_data']['x_test'].shape[-1])

  model = CNN_small(input_size)

  # optimizer and the loss function definition
  optimizer = torch.optim.Adam(model.parameters(),
                             lr = learning_rate,
                             weight_decay = weight_decay)
  criterion = torch.nn.CrossEntropyLoss()

  model.to(dev)
  criterion.to(dev)


  #******** Training loop *********
  best_model, train_losses, val_losses, train_accs, val_accs, info =\
      train_model(model, dat['dset_loaders'], dat['dset_sizes'],
                  criterion, optimizer, dev=dev, lr_scheduler=False,
                  num_epochs = num_epochs, verbose = verbose)


  test_samples = 50
  x_test = dat['test_data']['x_test'][:test_samples,:,:]
  y_test = dat['test_data']['y_test'][:test_samples]


  # predict test data
  preds = best_model(x_test.to(dev))
  preds_class = preds.data.max(1)[1]

  # get the accuracy
  corrects = torch.sum(preds_class == y_test.data.to(dev))
  test_acc = corrects.cpu().numpy()/x_test.shape[0]
  print("Test Accuracy :", test_acc)
  sum+=test_acc
  num+=1
  if test_acc>max:
    max=test_acc
  # save results
  tab = dict(Train_Acc= train_accs[info['best_epoch']],
            Val_Acc  = val_accs[info['best_epoch']],
            Test_Acc = test_acc, Epoch = info['best_epoch'] + 1)

  table.loc[description] = tab
  val_acc = np.max(val_accs)
  print('\n---------------------------------------------------------------------\n')

----------------Subject 1---------------------
Epoch 1/150
train loss: 0.8413, acc: 0.5174
val loss: 0.0505, acc: 0.6508
Epoch 2/150
train loss: 0.2993, acc: 0.5957
val loss: 0.0525, acc: 0.6349
Epoch 3/150
train loss: 0.1015, acc: 0.6884
val loss: 0.0711, acc: 0.6508
Epoch 4/150
train loss: 0.0357, acc: 0.8058
val loss: 0.0875, acc: 0.6032
Epoch 5/150
train loss: 0.0148, acc: 0.8594
val loss: 0.0551, acc: 0.6984
Epoch 6/150
train loss: 0.0129, acc: 0.8580
val loss: 0.0580, acc: 0.6508
Epoch 7/150
train loss: 0.0072, acc: 0.8986
val loss: 0.0411, acc: 0.6508
Epoch 8/150
train loss: 0.0031, acc: 0.9406
val loss: 0.0431, acc: 0.5873
Epoch 9/150
train loss: 0.0014, acc: 0.9609
val loss: 0.0412, acc: 0.6032
Epoch 10/150
train loss: 0.0014, acc: 0.9696
val loss: 0.0420, acc: 0.6190
Epoch 11/150
train loss: 0.0005, acc: 0.9899
val loss: 0.0503, acc: 0.6032
Epoch 12/150
train loss: 0.0024, acc: 0.9696
val loss: 0.0587, acc: 0.5556
Epoch 13/150
train loss: 0.0014, acc: 0.9783
val loss: 0.0449,

In [20]:
table.to_csv('results/CNN_small/Weibo2014_cnn_small.csv')

print(table)
print(f"\nAverage between subjects: {(sum/num):.2f}")
print(f"Max test_score between subjects: {max:.2f}")

           Train_Acc   Val_Acc  Test_Acc  Epoch
Subject1    1.000000  0.714286      0.58     67
Subject2    0.842029  0.634921      0.54      6
Subject3    0.914493  0.634921      0.62     12
Subject4    0.813043  0.555556      0.54      8
Subject5    0.485507  0.619048      0.36      1
Subject6    1.000000  0.777778      0.48     42
Subject7    0.915942  0.841270      0.78      6
Subject8    0.998551  0.603175      0.56     21
Subject9    0.962319  0.698413      0.58     68
Subject10   0.979710  0.507937      0.64     34

Average between subjects: 0.57
Max test_score between subjects: 0.78
